In [23]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torchaudio
import numpy as np
import pandas as pd
from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor, Wav2Vec2Config
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
import os
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import pickle


In [24]:
# Load class mapping
CLASS_MAPPING = {0: "Cry", 1: "Screaming", 2: "NotScreaming"}


In [25]:
# ✅ Force TensorFlow to run on CPU (Prevents GPU memory usage)
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Load YAMNet model
yamnet_model = hub.load("https://tfhub.dev/google/yamnet/1")

def extract_embedding(file_path):
    waveform, sr = librosa.load(file_path, sr=16000)
    waveform = waveform.astype(np.float32)
    scores, embeddings, spectrogram = yamnet_model(waveform)
    tf.keras.backend.clear_session()  # ✅ Free memory
    return np.mean(embeddings.numpy(), axis=0)

def get_yamnet_prediction(model,filename):
    embedding = extract_embedding(filename)
    embedding = np.expand_dims(embedding, axis=0)
    prediction = model.predict(embedding)
    predicted_class = np.argmax(prediction)
    
    print(f"Predicted Class: {CLASS_MAPPING[predicted_class]}")
    print(f"Prediction Scores: {prediction[0]}")
    
    answer = CLASS_MAPPING[predicted_class]
    probabilities = prediction[0]
    return answer, probabilities


In [26]:
# Set GPU device to visible again
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [27]:
class Wav2VecClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(Wav2VecClassifier, self).__init__()
        
        # Load the pre-trained Wav2Vec model
        self.wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
        
        # Freeze the feature extractor layers (optional)
        for param in self.wav2vec.feature_extractor.parameters():
            param.requires_grad = False
            
        # Get the output dimension of wav2vec
        hidden_size = self.wav2vec.config.hidden_size  # typically 768
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        # Get wav2vec features
        outputs = self.wav2vec(x)
        hidden_states = outputs.last_hidden_state
        
        # Pool the output (take mean of all time steps)
        pooled_output = torch.mean(hidden_states, dim=1)
        
        # Classify
        logits = self.classifier(pooled_output)
        return logits

# Let's test the model with random data
def test_model():
    # Create model instance
    model = Wav2VecClassifier(num_classes=3)
    model.eval()
    
    # Create random input tensor
    # Wav2Vec expects input shape: [batch_size, sequence_length]
    # Typical audio sampling rate is 16kHz, let's create 1 second of audio
    batch_size = 2
    sequence_length = 16000  # 1 second of audio at 16kHz
    random_audio = torch.randn(batch_size, sequence_length)
    
    # Forward pass
    with torch.no_grad():
        output = model(random_audio)
    
    print("Input shape:", random_audio.shape)
    print("Output shape:", output.shape)
    print("Output (logits):", output)

In [28]:
def preprocess_audio(audio_path):
    # Load audio
    waveform, sample_rate = torchaudio.load(audio_path)
    
    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    
    # Resample if necessary
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        waveform = resampler(waveform)
    
    # Convert to numpy array and squeeze
    waveform = waveform.squeeze().numpy()
    
    # Normalize audio
    waveform = waveform / np.max(np.abs(waveform))
    
    return waveform

def test_model_on_audio(model, audio_path, device):
    class_mapping = {
            'Cry': 0,
            'NotScreaming': 1,
            'Screaming': 2
        }
    
    # Preprocess audio
    waveform = preprocess_audio(audio_path)
    input_values = torch.FloatTensor(waveform).unsqueeze(0).to(device)
    
    # Forward pass
    model.eval()
    with torch.no_grad():
        outputs = model(input_values)
    
    # Get predicted class
    _, predicted = torch.max(outputs.data, 1)
    
    # Get class name
    class_mapping = {v: k for k, v in class_mapping.items()}
    predicted_class = class_mapping[predicted.item()]
    
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0).cpu().numpy()
    return predicted_class, probabilities


In [29]:
# Load Yamnet model
model_yamnet = tf.keras.models.load_model("yamnet_finetuned.h5")

# Load Wav2Vec model
model_wav2vec = Wav2VecClassifier(num_classes=3)
model_wav2vec.load_state_dict(torch.load('wav2vec_model.pth'))
model_wav2vec.eval()

# Test the model on test set in CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_wav2vec = model_wav2vec.to(device)

# Test our trained model on random audio file
filename = "Datasets/Cry/0c8f14a9-6999-485b-97a2-913c1cbf099c-1430760379259-1.7-m-26-hu.wav"

ans_yamnet, probabilities_yamnet = get_yamnet_prediction(model_yamnet,filename)
ans_wav2vec, probabilities_wav2vec = test_model_on_audio(model_wav2vec, filename, device)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted Class: Cry
Prediction Scores: [9.9248135e-01 7.2007808e-03 3.1792070e-04]


In [30]:
print(f"Yamnet Prediction: {ans_yamnet}")
print(f"Wav2Vec Prediction: {ans_wav2vec}")

Yamnet Prediction: Cry
Wav2Vec Prediction: Cry


In [31]:
print(f"Yamnet Probabilities: {probabilities_yamnet}")
print(f"Wav2Vec Probabilities: {probabilities_wav2vec}")

Yamnet Probabilities: [9.9248135e-01 7.2007808e-03 3.1792070e-04]
Wav2Vec Probabilities: [9.981421e-01 7.241565e-04 1.133691e-03]
